## Revenue Management

We consider the general case where the length of the buffer is $n$. Then the states are given by $\mathcal{X}:=A \cup B$, where $A= \{0,\ldots, n\}$ and $B= \{n+1,\ldots, 2n+1\}$, with $A$ denoting the states where the server is off and $B$ denoting the states where the server is on. The number of customers in the queue is exactly the state number for states in $A$ and the number of customers in the queue for states in $B$ is modulo $n+1$ of the state number. As for the action, we have $\mathcal{A}(x) = \{0,1\}$ where 0 (1) means the server is off (on).

For our exercise, we have `n=100`

With that, we can get evaluate the reward function by considering cases.

#### Server is switched off, $a = 0$

- $x <n$, $R(x,a,w) = (1-w)(-x)+w(-x-1)$
- $x =n$, $R(x,a,w) = (1-w)(-n)+w(-n-1000)$
- $n< x < 2n+1$, $R(x,a,w) = (1-w)(-(x \mod n+1))+w(-(x \mod n+1)-1)$
- $x =2n+1$, $R(x,a,w) = (1-w)(-n)+w(-n-1000)$

#### Server is switched off, $a = 1$

- $x <n$, $R(x,a,w) = (1-w)(-x)+w(-x-1)-10$
- $x =n$, $R(x,a,w) = (1-w)(-n)+w(-n-1)-10$
- $n< x < 2n+1$, $R(x,a,w) = (1-w)(-(x \mod n+1))+w(-(x \mod n+1)-1)$
- $x =2n+1$, $R(x,a,w) = (1-w)(-n)+w(-n-1)$


In [1]:
import numpy as np
from numpy.linalg import norm, inv

In [2]:
n = 100

In [3]:
def reward_function(x,a,w = 0.75,n = 100):
    """
    Computes the expected reward given current state x, action taken a, weight w and buffer length n.
    """
    if a == 0:
        if (x < n):
            r = (1-w)*(-x) + w*(-x-1)
        elif ((x == n) or (x == 2*n+1)):
            r = (1-w)*(-n) + w*(-n-1000);
        elif (x > n) and (x < (2*n+1) ):
            y = np.mod(x,n+1)
            r = (1-w)*(-y) + w*(-y-1)
    else:        
        if (x <= n):
            r = (1-w)*(-x) + w*(-x-1)-10
        elif (x > n) and (x < (2*n+1) ):
            y = np.mod(x,n+1)
            r = (1-w)*(-y) + w*(-y-1)
        else: 
            r = (1-w)*(-n) + w*(-n-1)  
        
    return r    

The function `reward_gen` generates two vectors `r0` and `r1` which gives us the expected rewards when we take $a=0$ and $a=1$ respectively.

In [4]:
def reward_gen(w = 0.75, n = 100):
    """
    Generates the rewards for action a=0 and a=1 given weight w and buffer length n.
    """
    R0 = np.zeros(2*(n+1))
    R1 = np.zeros(2*(n+1))
    for i in range(2*(n+1)):
        R0[i] = reward_function(i,0,w,n)
        R1[i] = reward_function(i,1,w,n)
        
    return R0, R1   

We generate the reward vectors `r0` and `r1` here.

In [5]:
R0, R1 = reward_gen()

We also have the discount factor $\alpha = 0.98$

In [6]:
alpha = .98

Taking the maximum over the possible action $\mathcal{A}=\{0,1\}$ over the entire state space $\mathcal{X}$, we get the reward vector `r`.

In [7]:
R = np.maximum(R0,R1)

In [8]:
def T(R, J, alpha = 0.98):
    """
    Applies the T operator:
    (TJ)(x) = max E[R(x,a,w) + alpha*J(f(x,a,w))|x]
    where r = max R(x,a,w) over action space
          alpha is the discount factor
          J is expected total reward
    """
    J = R + alpha*J
    
    return J
    

### Value iteration

We initialise the initial `J` to be all zeros and apply the operator `T` multiple times till the error is small

In [9]:
J = np.zeros(2*n+1+1)
err = 1
k = 0
while err > 0:
    J = T(R, J)
    err = norm((T(R,J)-J),1)
    k += 1

print ('Iterations:', k)    
print ('Error:', err)

Iterations: 1654
Error: 0.0


From above, we see that we needed 1654 iterations for $(TJ^{k})(x) = J(x)$

We would now like to derive a policy to based on the `J` values derived from value iteration

In [10]:
K = J.reshape(2,101)

In [11]:
np.argmax(K,axis = 0)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int64)

By comparing the the expected total reward for each state $x \in \mathcal{X}$ we see that the optimal policy is to keep the server off when there are less than 100 customers in the queue and to switch on the server when there are 100 customers in the queue. This policy can be explained by the huge cost incurred when a customer leaves when there is no more space in the buffer. Also the cost incurred when there are less than 100 customers in the queue is the same regardless of whether the server is on or off; the same cost in incurred whether we allow the customer to wait in the queue or when the server is switched on to serve the customer. The cost of 10 to switch on the server is also a deterance to switch on the server, unless we have a possibility of a large cost to be incurred; when there is no more space in the buffer.

### Policy Iteration

We initialise the initial stationary policy, `pi` to be switch off the server at every state

In [12]:
pi = np.zeros(2*(n+1))

#### Transition probability matrix

The transition probability matrix under policy `pi` is given by 

- if $0 \leq i \leq 100$

$$p_{ij}(\text{pi}(i)) =\begin{cases} 
0.25 &  j = i, pi(i) = 0, i <100\\
0.75 &  j = i+1, pi(i) = 0, i <100\\
1 & j=i, p(i) = 0, i = 100\\
1 & j=i + 101, p(i) = 1, i = 0\\
0.25 &  j = i + 100, pi(i) = 1, i < 100\\
0.75 &  j = i + 101, pi(i) = 1, i <100\\
0 & \text{otherwise}
\end{cases} $$

- if $101 \leq i \leq 201$

$$p_{ij}(\text{pi}(i)) =\begin{cases} 
0.25 &  j = i-101, pi(i) = 0, i <201\\
0.75 &  j = i-100, pi(i) = 0, i <201\\
1 & j=i-101, p(i) = 0, i = 201\\
1 & j=i, p(i) = 1, i = 101\\
0.25 &  j = i-1, pi(i) = 1, i < 201\\
0.75 &  j = i, pi(i) = 1, i <201\\
0 & \text{otherwise}
\end{cases} $$

In [13]:
def trans_mat_gen(pi, n = 100):
    """
    Generates transition probability matrix under the policy pi. The buffer is of length n.
    """
    P = np.zeros((2*(n+1),2*(n+1)))
    for i in range(n+1):
        if pi[i] == 0:
            P[i,i] = 0.25
            if i < n:
                P[i,i+1] = 0.75
            else:
                P[i,i] = 1
        else:
            P[i,i+n+1] = 0.75
            if i > 0:
                P[i,i+n] = 0.25
            else:
                P[i,i+n+1] = 1
    
    for i in range(n+1, 2*(n+1)):
        if pi[i] == 0:
            P[i,i-(n+1)] = 0.25
            if i < (2*n+1):
                P[i,i-n] = 0.75
            else:
                P[i,i-(n+1)] = 1
        else:
            P[i,i] = 0.75
            if i > (n+1):
                P[i,i-1] = 0.25
            else:
                P[i,i] = 1
         
    return P

A toy example where the buffer is 2 

In [14]:
pi1 = np.array([1,0,1,0,0,1])

In [15]:
trans_mat_gen(pi1,2)

array([[ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ],
       [ 0.  ,  0.25,  0.75,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.25,  0.75],
       [ 0.25,  0.75,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.25,  0.75,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.25,  0.75]])

#### Expected reward vector

The expected reward vector under policy `pi` is given to be

$$R_i = \mathbb{E}[R(i, \text{pi}(i), w)]$$

where $i$ is the $i-$th index of the reward vector.

- if pi(i) = 0, which means the server is switched off

    - for $0\leq i < n$, $R_i = (1-w)(-i)+w(-i-1)$
    - for $i = n$, $R_i = (1-w)(-n)+w(-n-1000)$
    - for $n+1\leq i < 2n+1$, $R_i = (1-w)(-(i-(n+1))) + w(-(i-(n+1))-1)$
    - for $i = 2n+1$, $R_i = (1-w)(-n)+w(-n-1000)$

- if pi(i) = 1, which means the server is switched on
    - for $0\leq i < n$, $R_i = (1-w)(-i)+w(-i-1)-10$
    - for $i = n$, $R_i = (1-w)(-n)+w(-n-1)-10$
    - for $n+1\leq i < 2n+1$, $R_i = (1-w)(-(i-(n+1))) + w(-(i-(n+1))-1)$
    - for $i = 2n+1$, $R_i = (1-w)(-n)+w(-n-1)$

In [16]:
def reward_vec(pi, w = 0.75, n = 100):
    """
    Takes in a vector pi that describes the policy, the weight w and returns the expected reward vector.
    """
    R = np.zeros(2*(n+1))
    for i in range(2*(n+1)):
        if pi[i] == 0:
            if (i < n):
                R[i] = (1-w)*(-i) + w*(-i-1)
            elif i == n:
                R[i] = (1-w)*(-n) + w*(-n-1000)
            elif (i >= (n+1)) and (i < 2*n+1):
                R[i] = (1-w)*(-(i-(n+1))) + w*(-(i-(n+1))-1)
            elif i == 2*n+1:
                R[i] = (1-w)*(-n) + w*(-n-1000)    
        else:
            if (i < n):
#                 R[i] = (1-w)*(-i) + w*(-i-1) - 10
                R[i] = (1-w)*(-(i-1)) + w*(-i) - 10 - 1     
            elif i == n:
                R[i] = (1-w)*(-n) + w*(-n-1) - 10
            elif (i >= (n+1)) and (i < 2*(n+1)):
                R[i] = (1-w)*(-(i-(n+1))) + w*(-(i-(n+1))-1)
            else:
                R[i] = (1-w)*(-n) + w*(-n-1)
            
    return R                     

In [17]:
def policy_eval(pi, w = 0.75, alpha = 0.98, n = 100):
    """
    Returns the corresponding reward function under the policy pi.
    """
    P = trans_mat_gen(pi, n)
    I = np.eye(P.shape[0])
    R = reward_vec(pi, w = w, n = n)
    
    return np.dot(inv(I-alpha*P),R)

In [18]:
def policy_improv(pi, alpha = 0.98, w = 0.75, n = 100):
    """
    Returns the new optimal policy pi based on the given transition probability matrix P (based on previous pi), 
    reward function J, discount factor alpha, weight w and buffer length n.
    """
    # compute the reward for actions
    R0, R1 = reward_gen(w = w, n = n)
    R = np.maximum(R0,R1) 
    
    pi0 = np.zeros(2*(n+1))
    pi1 = np.zeros(2*(n+1)) + 1
    
    # compute the transition probability matrix for the actions
    P0 = trans_mat_gen(pi0, n = n)
    P1 = trans_mat_gen(pi1, n = n)
        
    # compute the expectations for the actions
    J = policy_eval(pi, w = w, alpha = alpha, n = n)
    E0 = R0 + alpha * np.dot(P0,J)
    E1 = R1 + alpha * np.dot(P1,J)
    
    # stacking the expected rewards together and choosing the one that return better rewards
    E = np.vstack((E0,E1))
    new_pi = np.argmax(E, axis = 0)
    
    return new_pi

In [38]:
def policy_iter(pi, alpha = 0.98, w = 0.75, n = 100):
    """
    Performs policy iteration till policy reaches a stationary policy
    """
    k = 0
    while not np.array_equal(pi, policy_improv(pi, alpha = alpha, w = w, n = n)):
        pi = policy_improv(pi, alpha = alpha, w = w, n = n)
        k += 1
        
    print ('Iterations:', k)    
    return pi

In [24]:
# t = np.array([0,0,0,0,0,0])
t = np.random.randint(2, size = 6)
print (t)
n = 2

[0 1 1 0 0 1]


In [25]:
s = inv(np.eye(6) - .98*trans_mat_gen(t, n))
print (s)

[[  1.71229629   4.87769594   8.22242122   1.19503551  11.18696765
   22.80558339]
 [  0.39834517   5.01042236   8.44616057   1.22755348  11.49137493
   23.42614348]
 [  0.28155041   3.54136712   9.09014764   0.86763494  11.00700359
   25.21229629]
 [  0.71229629   4.87769594   8.22242122   2.19503551  11.18696765
   22.80558339]
 [  0.30453412   3.83045831   8.75056786   0.93846229  11.9055345
   24.27044293]
 [  0.28155041   3.54136712   8.09014764   0.86763494  11.00700359
   26.21229629]]


In [26]:
print (np.dot(inv(np.eye(6) - .98*trans_mat_gen(t, n)),reward_vec(t, w = .75, n = 2)))

[-246.62184448 -252.3122348  -246.96840617 -246.62184448 -245.08827606
 -236.96840617]


In [27]:
R0, R1 = reward_gen(w = 0.75, n = 2)
R = np.maximum(R0,R1) 
print ('R0:',R0)
print ('R1:',R1)
print (R)

R0: [ -7.50000000e-01  -1.75000000e+00  -7.52000000e+02  -7.50000000e-01
  -1.75000000e+00  -7.52000000e+02]
R1: [-10.75 -11.75 -12.75  -0.75  -1.75  -2.75]
[ -0.75  -1.75 -12.75  -0.75  -1.75  -2.75]


In [28]:
n = 2
t0 = np.zeros(2*(n+1))
t1 = np.zeros(2*(n+1)) + 1
print (t0)
print (t1)

[ 0.  0.  0.  0.  0.  0.]
[ 1.  1.  1.  1.  1.  1.]


In [29]:
P0 = trans_mat_gen(t0, n = n)
P1 = trans_mat_gen(t1, n = n)
print (P0)
print (P1)

[[ 0.25  0.75  0.    0.    0.    0.  ]
 [ 0.    0.25  0.75  0.    0.    0.  ]
 [ 0.    0.    1.    0.    0.    0.  ]
 [ 0.25  0.75  0.    0.    0.    0.  ]
 [ 0.    0.25  0.75  0.    0.    0.  ]
 [ 0.    0.    1.    0.    0.    0.  ]]
[[ 0.    0.    0.    1.    0.    0.  ]
 [ 0.    0.    0.    0.25  0.75  0.  ]
 [ 0.    0.    0.    0.    0.25  0.75]
 [ 0.    0.    0.    1.    0.    0.  ]
 [ 0.    0.    0.    0.25  0.75  0.  ]
 [ 0.    0.    0.    0.    0.25  0.75]]


In [30]:
J = policy_eval(t, w = 0.75, alpha = 0.98, n = 2)
print (J)

[-246.62184448 -252.3122348  -246.96840617 -246.62184448 -245.08827606
 -236.96840617]


In [31]:
E0 = R0 + alpha * np.dot(P0,J)
E1 = R1 + alpha * np.dot(P1,J)
print (E0)
print (E1)
E = np.vstack((E0,E1))

[-246.62184448 -245.08827606 -994.02903805 -246.62184448 -245.08827606
 -994.02903805]
[-252.43940759 -252.3122348  -246.96840617 -242.43940759 -242.3122348
 -236.96840617]


In [34]:
np.argmax(E, axis = 0)

array([0, 0, 1, 1, 1, 1], dtype=int64)

In [32]:
print (P0[0,:])
print (J)
print (np.dot(P0[0,:],J))

[ 0.25  0.75  0.    0.    0.    0.  ]
[-246.62184448 -252.3122348  -246.96840617 -246.62184448 -245.08827606
 -236.96840617]
-250.88963722


In [39]:
policy_iter(t, n = 2)

Iterations: 2


array([1, 1, 1, 1, 1, 1], dtype=int64)

In [33]:
print (P1[0,:])
print (J)
print (np.dot(P1[0,:],J))

[ 0.  0.  0.  1.  0.  0.]
[-246.62184448 -252.3122348  -246.96840617 -246.62184448 -245.08827606
 -236.96840617]
-246.621844475
